In [1]:
# Import the necessary libraries to work with tabulated data
# and for plotting.
using DataFrames
using StatPlots
using Plots
using Optim

INFO: Recompiling stale cache file /home/abieler/.julia/lib/v0.5/DataFrames.ji for module DataFrames.
INFO: Recompiling stale cache file /home/abieler/.julia/lib/v0.5/Plots.ji for module Plots.
INFO: Recompiling stale cache file /home/abieler/.julia/lib/v0.5/KernelDensity.ji for module KernelDensity.


In [7]:
# Lade Daten und speichere sie in der Variable "df" ab.
# df steht hier für DataFrame und ist vergleichbar mit
# einer Excel Tabelle.
# Jede Spalte der Tabelle ist dabei eine Variable.
# Jede Zeile der Tabelle ist eine "Messung"
# Geschlecht ist die Zielvariable welche wir voraussagen wollen.
# 1 := maennlich, 0 := weiblich

df_train = readtable("train.csv")
df_test = readtable("test.csv")
#raw_data  = convert(Matrix{Float64}, df[:, [:UmfangHandgelenk, :Alter]])

@show(size(df_train))
@show(size(df_test))
head(df_train)

size(df_train) = (891,12)
size(df_test) = (418,11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NA,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NA,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NA,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


In [8]:
# Darstellen der Messpunkte
scatter(df, :SibSp, :Age, group=:Survived)

# Wähle zwei Variablen aus die du denkst sind vielversprechend

INFO: Recompiling stale cache file /home/abieler/.julia/lib/v0.5/PyPlot.ji for module PyPlot.



 If the code has a Segmentation fault error switch to qt v4.8.5 by pasting the following code into julia: 
 
if !Plots.is_installed("PyPlot")
    Pkg.add("PyPlot")
end
withenv("PYTHON" => "") do
    Pkg.build("PyPlot")
end
import Conda
Conda.add("qt=4.8.5")

# now restart julia!


LoadError: MethodError: no method matching extractGroupArgs(::Symbol, ::StatsBase.#df, ::Symbol, ::Symbol)[0m
Closest candidates are:
  extractGroupArgs([1m[31m::AbstractArray{T,1}[0m, ::Any...) at /home/abieler/.julia/v0.5/Plots/src/args.jl:760
  extractGroupArgs{T,V<:AbstractArray{Int64,1}}([1m[31m::Dict{T,V<:AbstractArray{Int64,1}}[0m, ::Any...) at /home/abieler/.julia/v0.5/Plots/src/args.jl:772
  extractGroupArgs(::Symbol, [1m[31m::DataFrames.AbstractDataFrame[0m, ::Any...) at /home/abieler/.julia/v0.5/StatPlots/src/dataframes.jl:21[0m

In [4]:
# Normalisieren der Datenpunkte. Wird in einer späteren Lektion
# erklärt.

# X ist die Matrix der Messwerte
# y ist der Vektor "Geschlecht"

X_norm = (raw_data .- mean(raw_data, 1)) ./ std(raw_data, 1)

const X = hcat(ones(size(X_norm, 1)), X_norm)
const y = convert(Vector{Float64}, df[:,:Geschlecht]);

# Hypothese
Unser Ziel ist es, an hand der Messungen eine Vorhersage über das Geschlecht
der Person machen zu können. Das heisst für jede Messung wollen wir berechnen ob die Zielvariable = 1 (Mann) oder 0 (Frau) ist.

Man stellt mit Hilfe des Skalarprodukts eine Hypothese "H" auf um die Zielvariable y vorauszusagen.
H = dot(x, theta)

Unsere Hypothese ist also:

$$Geschlecht = y \approx H(s)$$

$$s = x_1\theta_1 + x_2 \theta_2 + \cdots + x_n \theta_n$$

Mit den unbekannten $\theta_i$ und den Messungen/Variablen $x_i$:

* $x_1$ = :UmfangBrust
* $x_2$ = :UmfangHuefte
* $\vdots$
* $x_7$ = :Groesse
* y = :Geschlecht


$s$ ist das Skalarprodukt zwischen $x$ und $\theta$. Zur Erinnerung:
$s$ kann eine Zahl zwischen -Unendlich bis +Unendlich sein.


Weil die Zielvariable nur entweder 0 oder 1 sein kann, brauchen wir also noch eine Funktion, die das Skalarprodukt nimmt und dann 0 oder 1 ausspuckt. Diese Funktion ist $H(s)$ (sprich: Ha von Ess)


Wir werden als erstes die folgende Funktion verwenden:

$H(s)$ = 1 falls s $\geq$ 0

$H(s)$ = 0 falls s $<$ 0

# Lösungsvorgehen
Nun müssen $\theta_1$ bis $\theta_7$ so gefunden werden, dass unsere
Hypothese möglichst gut funktioniert. Das ist Machine Learning.

![title](df_messung.png)

![title](df_variablen.png)

![title](Xy_new.png)

$$
X \cdot \theta = y
$$

Zeile für Zeile geschrieben:

$$
\theta_1 x_1 + \theta_2 x_2 + \dots + \theta_7 x_7 = y_1
$$

$$
\theta_1 \cdot \text{UmfangBrust}_{001} + \theta_2 \cdot \text{UmfangHuefte}_{001} + \dots + \theta_7 \cdot \text{Groesse}_{001} = \text{Geschlecht}_{001}
$$
$$
\theta_1 \cdot \text{UmfangBrust}_{002} + \theta_2 \cdot \text{UmfangHuefte}_{002} + \dots + \theta_7 \cdot \text{Groesse}_{002} = \text{Geschlecht}_{002}
$$
$$
\vdots
$$
$$
\theta_1 \cdot \text{UmfangBrust}_{507} + \theta_2 \cdot \text{UmfangHuefte}_{507} + \dots + \theta_7 \cdot \text{Groesse}_{507} = \text{Geschlecht}_{507}
$$

Jetzt muss man $\theta_1$ bis $\theta_7$ finden, so dass möglichst viele der Gleichungen oben stimmen.

# Optimierungsprobleme
Das finden von $\theta$ für unsere Aufgabe ist ein Optimierungsproblem. **Finde $\theta$ so dass $H(s)$ möglichst gut funktioniert!**. Es gitb sehr viele verschiedene Methoden um dieses Optimum zu finden und in der Regel können wir einfach eine dieser fertigen Methoden benutzen.

Als erstes brauchen wir also nur noch eine definition was überhaupt "gut" ist. Am liebsten wird diese Qualität als Skalar gemessen. Das Ziel ist dann diese Zahl möglichst klein/gross zu machen. Z.b.

* **Gewinn** einer Firma (mehr ist besser).
* **zu fahrende Kilometer** für einen Amazon Lieferwagen (weniger ist besser)


Julia hat ein Modul **Optim** welche verschiedene optimierungsverfahren bereitstellt.
Um damit Probleme zu lösen müssen wir eine Funktion definieren die ein $\theta$ annimmt,
und dann eine Zahl zurück gibt welche die Qualität von $\theta$ beschreibt.


1) bestimme einen Anfangswert für $\theta$.

2) Aus $\theta$ und $X$ bekommen wir das Skalarprodukt $s$.

3) H(s) ergibt unsere Vorhersage für "Geschlecht".

4) Vergleiche Vorhersage und "Geschlecht" um Qualität zu bestimmen.

5) Ändere $\theta$ so ab, dass Qualität zunimmt* --> (das macht Optim)

In [20]:
# Definition der Funktionen für optimize()
function H(s)
    if s >= 0
        return 1
    else
        return 0
    end
end

function cost(theta)
    s = X * theta
    prediction = H.(s)
    accuracy = mean(prediction .== y)
    score = -accuracy
    return score
end


cost (generic function with 1 method)

In [13]:
# Finde einen Anfangswert für theta
theta_init = randn(size(X, 2))

# Starte Optimierung
opt = optimize(cost, theta_init, LBFGS())

# Zeige Resultate
theta_opt = opt.minimizer
accuracy = opt.minimum
@show(accuracy)
@show(theta_opt);

accuracy = -0.48717948717948717
theta_opt = [1.22887,-0.220268,0.002]


![title](peaks.png)

![title](noisy_quadratic_surf.png)

![title](schwefelsfunction.jpg)

In [35]:
function plot_result(cost_func)
    xxmin, xxmax = (-3, 3)
    yymin, yymax = (-3, 3)
    nx = 100
    ny = 100

    xrange = linspace(xxmin, xxmax, nx)
    yrange = linspace(yymin, yymax, ny)

    z = zeros(nx*ny)

    ii = 1
    for ix in 1:nx
        for iy in 1:ny
            theta = [-1, xrange[ix], yrange[iy]]
            z[ii] = cost_func(theta)
            ii +=1 
        end
    end
    zplt = reshape(z, nx, ny)
    surface(xrange, yrange, zplt, levels=100)
end

plot_result (generic function with 1 method)

In [36]:
plot_result(cost)

Die Struktur des Optimierungsproblems ist mit der aktuellen Formulierung der cost_function nicht konvex. Konvex heisst "schüsselförmig". Dies hat zur Folge, dass Optimierungsalgorithmen das Optimum über Gradienten zu finden versuchen sehr leicht verwirrt werden und sich in ein lokales Optimum verirrent. Das Problem lässt sich beheben indem wir unsere cost_function umdefinieren, und zwar so, dass das Problem konvex wird.

Dazu verwenden wir die logistic function, manchmal auch logit genannt. Diese projiziert eine Zahl die zwischen -unendlich bis +unendlich liegt auf das Intervall [0 ... 1]. Diese Zahl kann dann als Wahrscheinlichkeit interpretiert werden zu einer bestimmten Klasse zu gehören. In unserem Fall also z.B. die Wahrscheinlichkeit zur Klasse "Mann" oder "Frau" zu gehören. Liegt diese Wahrscheinlichkeit über 50%, dann wird die entsprechende Klasse vorgeschlagen, ansonsten die andere Klasse.

In [23]:
function logit(s)
    return 1 ./ (1 .+ exp.(-s))
end

function cost_logistic_regression(theta)
    s = X * theta
    h = logit(s)
    score = sum(-y .* log.(h) .- (1 .- y) .* log.(1 .- h))
end

cost_logistic_regression (generic function with 1 method)

In [37]:
plot_result(cost_logistic_regression)